In [13]:
import requests
import time
import datetime
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()
COINDESKAPIKEY = os.getenv("COINDESKAPIKEY") 


In [17]:
import requests 

response = requests.get('https://data-api.coindesk.com/spot/v1/historical/hours',
    params={"market":"binance","instrument":"BTC-USDT","limit":10,"aggregate":1,"fill":"true","apply_mapping":"true","response_format":"JSON","to_ts":1740990845,"api_key":"4d50d36f37210bb124c685ada8aebfc92e6d384c35245c9eeee20dcf2b7377a0"},
    headers={"Content-type":"application/json; charset=UTF-8"}
)

json_response = response.json()

In [ ]:
def get_unix_timestamp(dt):
    """Convert a datetime object to a Unix timestamp (seconds)."""
    return int(dt.timestamp())

def fetch_hourly_data_batch(to_ts, limit=1500, market="binance", instrument="BTC-USDT"):
    """
    Fetch a batch of hourly data from the Coindesk API using the proper headers
    and parameters. This function returns the data from the "Data" key.
    """
    url = "https://data-api.coindesk.com/spot/v1/historical/hours"
    params = {
        "market": market,
        "instrument": instrument,
        "limit": limit,
        "aggregate": 1,           # hourly intervals
        "fill": "true",
        "apply_mapping": "true",
        "response_format": "JSON",
        "to_ts": to_ts,
        "api_key": COINDESKAPIKEY
    }
    headers = {"Content-type": "application/json; charset=UTF-8"}
    
    response = requests.get(url, params=params, headers=headers)
    response.raise_for_status()  # Raise an exception for HTTP errors
    data = response.json()
    # Return the array of records from the "Data" key per the schema
    return data.get("Data", [])

def fetch_hourly_data_npages():
    """
    Paginate backwards using the to_ts parameter for a limited number of pages.
    Here, we limit the calls to 3 pages. The pagination uses the "TIMESTAMP" field.
    """
    # Set the end time to now (UTC)
    end_dt = datetime.datetime.utcnow()
    end_ts = get_unix_timestamp(end_dt)
    print("End timestamp:", end_ts)
    
    all_records = []
    current_to_ts = end_ts
    max_pages = 6  # Limit to 6 API calls
    page_count = 0

    while page_count < max_pages:
        batch_data = fetch_hourly_data_batch(to_ts=current_to_ts)
        
        if not batch_data:
            print("No more data returned. Stopping pagination.")
            break
        
        all_records.extend(batch_data)
        page_count += 1
        
        # Using the first element as the earliest record based on the JSON schema
        earliest_record = batch_data[0]
        print("Earliest record on page:", earliest_record)
        earliest_timestamp = int(earliest_record["TIMESTAMP"])
        
        print(f"Page {page_count} retrieved, earliest timestamp in batch: {earliest_timestamp}")
        
        # Set the new to_ts to one hour before the earliest timestamp to avoid duplicates
        current_to_ts = earliest_timestamp - 3600
        time.sleep(1)  # Pause to avoid rate limiting
    
    df = pd.DataFrame(all_records)
    
    if df.empty:
        print("No data retrieved.")
    else:
        # Verify that the 'TIMESTAMP' column exists
        if "TIMESTAMP" not in df.columns:
            print("ERROR: The DataFrame does not have a 'TIMESTAMP' column. Columns are:", df.columns)
        else:
            # Convert the TIMESTAMP column to a datetime for easier interpretation
            df["datetime"] = pd.to_datetime(df["TIMESTAMP"], unit='s', utc=True)
            df.sort_values(by="datetime", inplace=True)
    
    return df




In [26]:
# Execute and display results
df_prices = fetch_hourly_data_npages()
if not df_prices.empty:
    print("Fetched data shape:", df_prices.shape)
    display(df_prices.head())
else:
    print("No data retrieved.")

End timestamp: 1741028246
Earliest record on page: {'UNIT': 'HOUR', 'TIMESTAMP': 1735628400, 'TYPE': '954', 'MARKET': 'binance', 'INSTRUMENT': 'BTCUSDT', 'MAPPED_INSTRUMENT': 'BTC-USDT', 'BASE': 'BTC', 'QUOTE': 'USDT', 'BASE_ID': 1, 'QUOTE_ID': 7, 'TRANSFORM_FUNCTION': '', 'OPEN': 92816, 'HIGH': 92910.03, 'LOW': 92770, 'CLOSE': 92860.75, 'FIRST_TRADE_TIMESTAMP': 1735628400, 'LAST_TRADE_TIMESTAMP': 1735631999, 'FIRST_TRADE_PRICE': 92816, 'HIGH_TRADE_PRICE': 92910.03, 'HIGH_TRADE_TIMESTAMP': 1735631761, 'LOW_TRADE_PRICE': 92770, 'LOW_TRADE_TIMESTAMP': 1735628463, 'LAST_TRADE_PRICE': 92860.75, 'TOTAL_TRADES': 61503, 'TOTAL_TRADES_BUY': 33711, 'TOTAL_TRADES_SELL': 27792, 'TOTAL_TRADES_UNKNOWN': 0, 'VOLUME': 485.23565, 'QUOTE_VOLUME': 45059075.3090525, 'VOLUME_BUY': 282.11043, 'QUOTE_VOLUME_BUY': 26197557.4775072, 'VOLUME_SELL': 203.12522, 'QUOTE_VOLUME_SELL': 18861517.8315453, 'VOLUME_UNKNOWN': 0, 'QUOTE_VOLUME_UNKNOWN': 0}
Page 1 retrieved, earliest timestamp in batch: 1735628400
Earliest

,UNIT,TIMESTAMP,TYPE,MARKET,INSTRUMENT,MAPPED_INSTRUMENT,BASE,QUOTE,BASE_ID,QUOTE_ID,...,TOTAL_TRADES_UNKNOWN,VOLUME,QUOTE_VOLUME,VOLUME_BUY,QUOTE_VOLUME_BUY,VOLUME_SELL,QUOTE_VOLUME_SELL,VOLUME_UNKNOWN,QUOTE_VOLUME_UNKNOWN,datetime
7500,HOUR,1708628400,954,binance,BTCUSDT,BTC-USDT,BTC,USDT,1,7,...,0,1412.76654,7.286829e+07,602.38295,3.106344e+07,810.38359,4.180485e+07,0,0,2024-02-22 19:00:00+00:00
7501,HOUR,1708632000,954,binance,BTCUSDT,BTC-USDT,BTC,USDT,1,7,...,0,3121.92839,1.618850e+08,1779.83133,9.229998e+07,1342.09706,6.958498e+07,0,0,2024-02-22 20:00:00+00:00
7502,HOUR,1708635600,954,binance,BTCUSDT,BTC-USDT,BTC,USDT,1,7,...,0,1264.97024,6.538372e+07,667.58788,3.450595e+07,597.38236,3.087777e+07,0,0,2024-02-22 21:00:00+00:00
7503,HOUR,1708639200,954,binance,BTCUSDT,BTC-USDT,BTC,USDT,1,7,...,0,1357.06045,6.981523e+07,695.54171,3.578757e+07,661.51874,3.402766e+07,0,0,2024-02-22 22:00:00+00:00
7504,HOUR,1708642800,954,binance,BTCUSDT,BTC-USDT,BTC,USDT,1,7,...,0,1038.76395,5.331609e+07,535.06306,2.746374e+07,503.70089,2.585235e+07,0,0,2024-02-22 23:00:00+00:00


In [27]:
# Save to CSV if desired
df_prices.to_csv("../data/raw/btc_usdt_hourly.csv", index=False)